In [1]:
#pip install azure-storage-file-datalake pandas azure-storage-blob azure-functions python-dotenv langchain-openai langchain pinecone-client pyarrow adlfs fsspec langchain_pinecone -q

# importar librerias y modulos de codigo

In [1]:
import pandas as pd
from dotenv import load_dotenv
import pandas as pd
import tiktoken

load_dotenv()

True

In [3]:
from external.pinecone_vdb import PineconeVectorDB
from external.open_ia_chat import OpenAIChatModel
from external.lang_chain_manager import LangChainManager

from app.repositories.pinecone_vdb_loader import PinceconeVDBRepository
from app.repositories.chat_model import ChatModelRepository
from app.repositories.agent_manager import AgentManagerRepository

# Cargar y dividir documentos, Crear embeddings y cargar documentos en base de datos vectorial

In [4]:
langchain_manager = LangChainManager()
pinecone_vdb = PineconeVectorDB()
open_ia_model = OpenAIChatModel(temperature=0.1)


vdb_repository= PinceconeVDBRepository(pinecone_vdb,langchain_manager)
chat_repository = ChatModelRepository(open_ia_model,langchain_manager)
agent_repository =AgentManagerRepository(open_ia_model,langchain_manager)


## crear documentos con datos procesados en zona curated

In [5]:
df_curated = pd.read_csv('df_curated_videos_youtube.csv',sep=';')
df_curated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288 entries, 0 to 3287
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   chanel_name        3288 non-null   object 
 1   video_id           3288 non-null   object 
 2   source             3288 non-null   object 
 3   publish_date       3288 non-null   object 
 4   duration           3288 non-null   float64
 5   last_update_date   3288 non-null   object 
 6   title              3288 non-null   object 
 7   text               3288 non-null   object 
 8   year               3288 non-null   int64  
 9   numero_tokens      3288 non-null   int64  
 10  numero_caracteres  3288 non-null   int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 282.7+ KB


In [6]:
#crear documentos 
docs=vdb_repository.create_documents_from_df(df_curated)

In [7]:
len(docs)

3288

In [8]:
docs[0]

Document(page_content='analisis tecnico bancolombia $cib 09 sep 14. muy buenos días vamos a hacer un análisis breve de la situación de bancolombia el día de ayer habíamos comentado perdón y al twitter privado que la grafica bancolombia y desde ahí ve en eeuu estaba en palabras mías horrible estaba rompiendo a la baja un hombro cabeza hombro se desmonten twitter este fue el tweet es el hombro cabeza hombro esta fue la ruptura se los advertimos cuando estaba 61 33 igual ya les había comentado que yo venía corto es de 64 dólares les había comentado el corto lo pueden encontrar también en la cuenta de twitter ahí está la señal de corto que les envié cuando abrí mi corto personal y ese corto nos cerramos hoy en 59 95 con una rentabilidad del 6.7 por ciento ya es la tercera el tercer trade que les he comentado como cuestión personal no en bancolombia sino en si ve que hemos hecho en eeuu todos positivos razón para decirlo en eeuu es que no tenemos ese efecto del dólar que como les había come

In [9]:
print(docs[-1].page_content)

bitcoin explotando | crypto | btc. yo que iba a pasar primero de enero metido en cama descansando cómo les va mis amigos les saludo gre kiroga bitcoin explotando la locura continúa 44795 qué es hecho querido miren cómo les va no los veo dónde están que no los veo ah veo la primera aparición por ahí alguien por twitter lo hemos estado siguiendo verdad por twitter les hemos desde antes que rompiera ya les estaba hablando de de este tema ayer hicimos un video cómo están bienvenidos todos ayer hicimos un video donde les hacía una comparación con el pasado en un evento exactamente igual a lo que estamos viendo el día de hoy feliz año para todos gracias por estar conectados bueno varias cosas les voy tres cosas te voy a decir primero bienvenidos a este streaming extraordinario así sin bañarme eh para con las calletanas que les mandan feliz año a todos bitcoin arrancó el año bien okay primer punto importante segundo yo ya voy comprado pase lo que pase tercero si alguien va haciendo un trade a

In [10]:
print(docs[-1].metadata)

{'chanel_name': 'USACRYPTONOTICIAS', 'video_id': 'fpfy6LOVvt0', 'source': 'https://www.youtube.com/watch?v=fpfy6LOVvt0', 'publish_date': '2024-01-01', 'duration': 2090.0, 'last_update_date': '2024-06-01', 'title': 'bitcoin explotando | crypto | btc', 'year': 2024, 'numero_tokens': 6638, 'numero_caracteres': 24430}


In [11]:
print(docs[-200].page_content)

valkyrie dice que el jueves comenzará a operar el etf de bitcoin. saludos cordiales pues ya lo estamos viendo dice valkirie que espera que el visto bueno de la sec para el etf de bitcoin se apruebe mañana y que la negociación podría comenzar el jueves desde que ha saltado esta noticia bueno hay cierta expectación en el mercado la podemos valorar esta la estamos viendo en investing pero ha sido publicada en diversos en diversos medios y es verdad que la comunidad inversora está con muchísima impaciencia sobre esta decisión bueno como la puedes encontrar en en investing y sabes que mi norma no es rellenar minutos para entretenerte sino informarte y formarte como la puedes encontrar tanto en investing como en en otras plataformas mira lo que ha estado sucediendo la cuestión es pero no está ya por descontado no se da por descontado ya voy a ver si aparezco aquí un poco en grande tengo desajustada la cámara los colores porque sabes que estoy utilizando la pantalla interactiva pero no import

In [13]:
print(docs[-200].metadata)

{'chanel_name': 'Bitcoin hoy', 'video_id': 'BvA-ZTgqyL8', 'source': 'https://www.youtube.com/watch?v=BvA-ZTgqyL8', 'publish_date': '2024-01-09', 'duration': 513.0, 'last_update_date': '2024-06-01', 'title': 'valkyrie dice que el jueves comenzará a operar el etf de bitcoin', 'year': 2024, 'numero_tokens': 1837, 'numero_caracteres': 7498}


## Dividir documentos en chunks de 300 tokens con solapamiento de 30 tokens

In [14]:
#dividir documentos
splitted_docs=vdb_repository.split_documents(docs,chunk_size=300,chunk_overlap=30)

In [15]:
len(splitted_docs)

85942

In [16]:
splitted_docs[0].page_content

'analisis tecnico bancolombia $cib 09 sep 14. muy buenos días vamos a hacer un análisis breve de la situación de bancolombia el día de ayer habíamos comentado perdón y al twitter privado que la grafica bancolombia y desde ahí ve en eeuu estaba en palabras mías horrible estaba rompiendo a la baja un hombro cabeza hombro se desmonten twitter este fue el tweet es el hombro cabeza hombro esta fue la ruptura se los advertimos cuando estaba 61 33 igual ya les había comentado que yo venía corto es de 64 dólares les había comentado el corto lo pueden encontrar también en la cuenta de twitter ahí está la señal de corto que les envié cuando abrí mi corto personal y ese corto nos cerramos hoy en 59 95 con una rentabilidad del 6.7 por ciento ya es la tercera el tercer trade que les he comentado como cuestión personal no en bancolombia sino en si ve que hemos hecho en eeuu todos positivos razón para decirlo en eeuu es que no tenemos ese efecto del dólar que como les había comentado el dólar está af

## Agregar ids a documentos divididos y modificar metadata para cargar en base de datos vectorial.

In [17]:
process_docs,ids = langchain_manager.add_ids_to_documents(splitted_docs,id_column='video_id')


## Crear dataframe con nueva metadata (particion de documentos) e IDs

In [18]:
data = [ d.metadata for d in process_docs]

In [19]:
# Crear DataFrame
df = pd.DataFrame(data, index=ids)
df

,chanel_name,video_id,source,publish_date,duration,last_update_date,title,year,numero_tokens,numero_caracteres,split_tokens,part
I1PG0G4Ix7w-0,ARENA ALFA,I1PG0G4Ix7w,https://www.youtube.com/watch?v=I1PG0G4Ix7w,2014-09-22,241.0,2024-06-01,analisis tecnico bancolombia $cib 09 sep 14,2014,946,3579,300,0
I1PG0G4Ix7w-1,ARENA ALFA,I1PG0G4Ix7w,https://www.youtube.com/watch?v=I1PG0G4Ix7w,2014-09-22,241.0,2024-06-01,analisis tecnico bancolombia $cib 09 sep 14,2014,946,3579,300,1
I1PG0G4Ix7w-2,ARENA ALFA,I1PG0G4Ix7w,https://www.youtube.com/watch?v=I1PG0G4Ix7w,2014-09-22,241.0,2024-06-01,analisis tecnico bancolombia $cib 09 sep 14,2014,946,3579,300,2
I1PG0G4Ix7w-3,ARENA ALFA,I1PG0G4Ix7w,https://www.youtube.com/watch?v=I1PG0G4Ix7w,2014-09-22,241.0,2024-06-01,analisis tecnico bancolombia $cib 09 sep 14,2014,946,3579,136,3
uRnB41tY8RA-0,ARENA ALFA,uRnB41tY8RA,https://www.youtube.com/watch?v=uRnB41tY8RA,2014-09-22,172.0,2024-06-01,video analisis tecnico avianca 05 sept 14,2014,621,2449,300,0
...,...,...,...,...,...,...,...,...,...,...,...,...
fpfy6LOVvt0-20,USACRYPTONOTICIAS,fpfy6LOVvt0,https://www.youtube.com/watch?v=fpfy6LOVvt0,2024-01-01,2090.0,2024-06-01,bitcoin explotando | crypto | btc,2024,6638,24430,300,20
fpfy6LOVvt0-21,USACRYPTONOTICIAS,fpfy6LOVvt0,https://www.youtube.com/watch?v=fpfy6LOVvt0,2024-01-01,2090.0,2024-06-01,bitcoin explotando | crypto | btc,2024,6638,24430,300,21
fpfy6LOVvt0-22,USACRYPTONOTICIAS,fpfy6LOVvt0,https://www.youtube.com/watch?v=fpfy6LOVvt0,2024-01-01,2090.0,2024-06-01,bitcoin explotando | crypto | btc,2024,6638,24430,300,22
fpfy6LOVvt0-23,USACRYPTONOTICIAS,fpfy6LOVvt0,https://www.youtube.com/watch?v=fpfy6LOVvt0,2024-01-01,2090.0,2024-06-01,bitcoin explotando | crypto | btc,2024,6638,24430,300,23


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85942 entries, I1PG0G4Ix7w-0 to fpfy6LOVvt0-24
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   chanel_name        85942 non-null  object 
 1   video_id           85942 non-null  object 
 2   source             85942 non-null  object 
 3   publish_date       85942 non-null  object 
 4   duration           85942 non-null  float64
 5   last_update_date   85942 non-null  object 
 6   title              85942 non-null  object 
 7   year               85942 non-null  int64  
 8   numero_tokens      85942 non-null  int64  
 9   numero_caracteres  85942 non-null  int64  
 10  split_tokens       85942 non-null  int64  
 11  part               85942 non-null  int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 8.5+ MB


In [21]:
df.to_csv('dataset_videos_youtube_300_tokens.csv',index=True,sep =';')

## Cargar documentos en base de datos vectorial

In [22]:
from tqdm import tqdm
import time

index_name='youtube-transcription-300-tokens-index'
batch_size = 200  
num_batches = len(ids) // batch_size + (1 if len(ids) % batch_size > 0 else 0)

batch_ids_loaded = []

for batch_index in tqdm(range(num_batches), desc="Processing batches"):
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, len(ids))
    batch_ids = ids[start_index:end_index]
    batch_process_docs = process_docs[start_index:end_index]
   
    try:
        batch_id_loaded = langchain_manager.add_documents_vectostore(
            index_name,
            batch_process_docs,
            batch_ids
        )
    
    except Exception as e:
        print("Error processing batch:", e)
        time.sleep(10)
        # Retry processing batch
        batch_id_loaded = langchain_manager.add_documents_vectostore(
            index_name,
            batch_process_docs,
            batch_ids
        )

    batch_ids_loaded.extend(batch_id_loaded)
    
    time.sleep(1) 

Processing batches:   0%|          | 0/430 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 430/430 [7:22:49<00:00, 61.79s/it]  


# Recuperar documetos

In [23]:
from external.pinecone_vdb import PineconeVectorDB
from external.lang_chain_manager import LangChainManager
from app.repositories.pinecone_vdb_retriever import PinceconeVDBRepository

## Recuperar documentos de base de datos vectorial por similud vector

In [24]:
pc_vdb =PineconeVectorDB()
lang_chain = LangChainManager()
vdb_retriever = PinceconeVDBRepository(pc_vdb,lang_chain)

In [25]:
#pc_vdb.list_indexes()
index_name='youtube-transcription-300-tokens-index'
vdb_retriever.describe_index_vdb(index_name)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 85942}},
 'total_vector_count': 85942}

In [26]:
query_test = 'patron de caida en el gráfico de 4 horas de bitcoin'
list_retriever = vdb_retriever.query_documents_vdb(query= query_test,index_name=index_name, n_documents=3,metadata_filter={"publish_date": "2024-05-08"})
list_retriever

[Document(page_content='¿rebote de hoy en btc puede tener continuidad alcista?. saludos cordiales bitcoin en la serie de 4 horas lo estamos viendo se ha trasladado la presión correctiva la presión bajista los 62070 este nivel que ya vimos en el primer análisis de esta mañana en 4 horas ha estado el precio detenido en los 6338 hasta que ha caído la presión se ha venido a los 62,70 bueno vamos a darle una vuelta de tuerca a todo vamos a actualizar esta información serie diaria 4 horas intr diarios y vamos a dejarlo ya preparado con qué proyección puede tener para finalizar la jornada para la jornada de mañana jueves y lo que vamos llevando ya de [música] semana soy esteban pérez si es la primera vez que estás viendo el canal de bitcoin hoy hacemos el análisis s técnico con la acción del precio de niveles calculados también desde el punto de vista fundamental con aquellos datos que se puedan publicar en el calendario macroeconómico en fin situaciones que se puedan ir dando que puedan move

In [27]:
len(list_retriever)

3

In [28]:
list_retriever[0].page_content

'¿rebote de hoy en btc puede tener continuidad alcista?. saludos cordiales bitcoin en la serie de 4 horas lo estamos viendo se ha trasladado la presión correctiva la presión bajista los 62070 este nivel que ya vimos en el primer análisis de esta mañana en 4 horas ha estado el precio detenido en los 6338 hasta que ha caído la presión se ha venido a los 62,70 bueno vamos a darle una vuelta de tuerca a todo vamos a actualizar esta información serie diaria 4 horas intr diarios y vamos a dejarlo ya preparado con qué proyección puede tener para finalizar la jornada para la jornada de mañana jueves y lo que vamos llevando ya de [música] semana soy esteban pérez si es la primera vez que estás viendo el canal de bitcoin hoy hacemos el análisis s técnico con la acción del precio de niveles calculados también desde el punto de vista fundamental con aquellos datos que se puedan publicar en el calendario macroeconómico en fin situaciones que se puedan ir dando que puedan mover el precio también de 

In [29]:
list_retriever[0].metadata

{'chanel_name': 'Bitcoin hoy',
 'duration': 468.0,
 'last_update_date': '2024-06-01',
 'numero_caracteres': 5875.0,
 'numero_tokens': 1467.0,
 'part': 0.0,
 'publish_date': '2024-05-08',
 'source': 'https://www.youtube.com/watch?v=IeTGexDPs9E',
 'split_tokens': 300.0,
 'title': '¿rebote de hoy en btc puede tener continuidad alcista?',
 'video_id': 'IeTGexDPs9E',
 'year': 2024.0}

## Recuperar documentos de base de datos vectorial por ID

In [30]:
list_retriever = vdb_retriever.query_ids_vdb(ids= ['IeXyurF4uyw-6'],index_name=index_name)
list_retriever

[{'id': 'IeXyurF4uyw-6',
  'metadata': {'chanel_name': 'USACRYPTONOTICIAS',
               'duration': 6596.0,
               'last_update_date': '2024-06-01',
               'numero_caracteres': 77940.0,
               'numero_tokens': 20627.0,
               'part': 6.0,
               'publish_date': '2023-01-11',
               'source': 'https://www.youtube.com/watch?v=IeXyurF4uyw',
               'split_tokens': 300.0,
               'text': ' que estoy diciéndoles sin stop loss por una razón '
                       'porque voy a 3x porque voy a 3x y mi precio de '
                       'liquidación está en 11580 podría caer hasta allá sí '
                       'pero para eso tengo una orden de compra en 12000 que me '
                       'promediar mucho más abajo por allá en 000 y si veo que '
                       'llega a 8000 por ejemplo o a 12000 pondré otra orden de '
                       'compra por allá en los 000 para promediar hacia abajo '
                  

In [31]:
list_retriever[0]['id']

'IeXyurF4uyw-6'

In [32]:
list_retriever[0]['metadata']

{'chanel_name': 'USACRYPTONOTICIAS',
 'duration': 6596.0,
 'last_update_date': '2024-06-01',
 'numero_caracteres': 77940.0,
 'numero_tokens': 20627.0,
 'part': 6.0,
 'publish_date': '2023-01-11',
 'source': 'https://www.youtube.com/watch?v=IeXyurF4uyw',
 'split_tokens': 300.0,
 'text': ' que estoy diciéndoles sin stop loss por una razón porque voy a 3x porque voy a 3x y mi precio de liquidación está en 11580 podría caer hasta allá sí pero para eso tengo una orden de compra en 12000 que me promediar mucho más abajo por allá en 000 y si veo que llega a 8000 por ejemplo o a 12000 pondré otra orden de compra por allá en los 000 para promediar hacia abajo pero si se dan cuenta no me he enloquecido mis amigos no estoy hablando de 000 de 000 no parciales solo compré el 0.019 de un bitcoin y ese potenciamiento me tienen $29 pero yo solo invertí $110 hoy ya van en 117 que puede caer más sí mis amigos pero así funciona la acumulación así funciona el acumular ya entonces no nos confundamos que de

## Agregar embeddings a df historico 

In [37]:
df_documentos = pd.read_csv('dataset_videos_youtube_300_tokens.csv',sep =';')
df_documentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85942 entries, 0 to 85941
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         85942 non-null  object 
 1   chanel_name        85942 non-null  object 
 2   video_id           85942 non-null  object 
 3   source             85942 non-null  object 
 4   publish_date       85942 non-null  object 
 5   duration           85942 non-null  float64
 6   last_update_date   85942 non-null  object 
 7   title              85942 non-null  object 
 8   year               85942 non-null  int64  
 9   numero_tokens      85942 non-null  int64  
 10  numero_caracteres  85942 non-null  int64  
 11  split_tokens       85942 non-null  int64  
 12  part               85942 non-null  int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 8.5+ MB


In [ ]:
from tqdm import tqdm
import time

batch_size = 2000
num_batches = len(df_documentos) // batch_size + 1  # Calculate the number of batches

batch_doc_retriever = []

ids = df_documentos['Unnamed: 0']

for batch_index in tqdm(range(num_batches), desc="Processing batches"):
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, len(df_documentos))
    batch_ids = ids[start_index:end_index]
   
    try:
        list_retriever = vdb_retriever.query_ids_vdb(ids=batch_ids, index_name=index_name)
    except Exception as e:
        print("Error processing batch:", e)
        time.sleep(5)  # Retry processing batch after waiting for 10 seconds
        list_retriever = vdb_retriever.query_ids_vdb(ids=batch_ids, index_name=index_name)
    
    batch_doc_retriever.extend(list_retriever)

    time.sleep(1)  # Add a short delay between batches

In [39]:
new_batch_doc_retriever =[]
for doc in batch_doc_retriever:
    new_doc_retriever = doc['metadata']
    new_doc_retriever['id'] = doc['id']
    new_doc_retriever['embedding'] = str(doc['values'])
    new_batch_doc_retriever.append(new_doc_retriever)



In [40]:
df_with_embeddings = pd.DataFrame(new_batch_doc_retriever)
df_with_embeddings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   chanel_name        80000 non-null  object 
 1   duration           80000 non-null  float64
 2   last_update_date   80000 non-null  object 
 3   numero_caracteres  80000 non-null  float64
 4   numero_tokens      80000 non-null  float64
 5   part               80000 non-null  float64
 6   publish_date       80000 non-null  object 
 7   source             80000 non-null  object 
 8   split_tokens       80000 non-null  float64
 9   text               80000 non-null  object 
 10  title              80000 non-null  object 
 11  video_id           80000 non-null  object 
 12  year               80000 non-null  float64
 13  id                 80000 non-null  object 
 14  embedding          80000 non-null  object 
dtypes: float64(6), object(9)
memory usage: 9.2+ MB


In [41]:
df_with_embeddings.to_csv('dataset_videos_youtube_embeddings_300_tokens.csv',sep =';',index=False)